# CodTech Internship Task 4 - Generative Text Model
This notebook demonstrates text generation using two approaches:
- Pretrained GPT-2 (via Hugging Face Transformers)
- Custom LSTM model (using TensorFlow/Keras)

In [ ]:
!pip install transformers torch tensorflow tqdm

## 🧠 GPT-2 Based Text Generation

In [ ]:

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

prompt = "The future of artificial intelligence is"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=100, num_return_sequences=1)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    

## 🔁 LSTM Based Text Generation (Simple Example)

In [ ]:

import tensorflow as tf
import numpy as np

text = "hello world hello world hello world"
chars = sorted(list(set(text)))
char2idx = {u:i for i, u in enumerate(chars)}
idx2char = np.array(chars)
text_as_int = np.array([char2idx[c] for c in text])

seq_length = 5
examples_per_epoch = len(text)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target).batch(1, drop_remainder=True)

vocab_size = len(chars)
embedding_dim = 8
rnn_units = 64

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[1, None]),
    tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])

model.compile(optimizer='adam', loss=lambda labels, logits: tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True))

model.fit(dataset, epochs=10)
    

In [ ]:

# Text generation from trained LSTM
input_eval = [char2idx[s] for s in "hello"]
input_eval = tf.expand_dims(input_eval, 0)
text_generated = []
temperature = 1.0

model.reset_states()
for i in range(20):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])

print("Generated:", "hello" + ''.join(text_generated))
    